# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### My assumption is there are (5) 1-year long campaigns from May 2008 to June 2013 represented in this data.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import time

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
)

In [3]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



CHECKING DATA HEALTH

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [6]:
# Since we might be checking for missing data repeatedly
def summarize_missing_values(dframe):
    counts_of_missing_per_feature = dframe.isnull().sum()
    percentages_missing_per_feature = (counts_of_missing_per_feature / len(dframe)) * 100

    missing_summary = pd.DataFrame({
                                    'missing_count': counts_of_missing_per_feature ,
                                    'missing_percent': percentages_missing_per_feature.round(2)
                                })

    missing_summary.sort_values(by='missing_percent', ascending=False, inplace=True)
    print(missing_summary)

In [7]:
# Is the raw data like swiss cheese
summarize_missing_values(df)

                missing_count  missing_percent
age                         0              0.0
campaign                    0              0.0
nr.employed                 0              0.0
euribor3m                   0              0.0
cons.conf.idx               0              0.0
cons.price.idx              0              0.0
emp.var.rate                0              0.0
poutcome                    0              0.0
previous                    0              0.0
pdays                       0              0.0
duration                    0              0.0
job                         0              0.0
day_of_week                 0              0.0
month                       0              0.0
contact                     0              0.0
loan                        0              0.0
housing                     0              0.0
default                     0              0.0
education                   0              0.0
marital                     0              0.0
y            

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

### The local task is to predict whether or not a customer will purchase a certificate of deposit of any term length.  The global business objective is to increase the effectiveness of those telemarketing campaigns targeting term purchases.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

DROP "duration" AND OTHER NON-BANKING RELATED FEATURES

In [8]:
df_lite = df.drop(columns=["age", "job", "marital", "education", "month", "day_of_week", "duration"])


In [9]:
df_lite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   default         41188 non-null  object 
 1   housing         41188 non-null  object 
 2   loan            41188 non-null  object 
 3   contact         41188 non-null  object 
 4   campaign        41188 non-null  int64  
 5   pdays           41188 non-null  int64  
 6   previous        41188 non-null  int64  
 7   poutcome        41188 non-null  object 
 8   emp.var.rate    41188 non-null  float64
 9   cons.price.idx  41188 non-null  float64
 10  cons.conf.idx   41188 non-null  float64
 11  euribor3m       41188 non-null  float64
 12  nr.employed     41188 non-null  float64
 13  y               41188 non-null  object 
dtypes: float64(5), int64(3), object(6)
memory usage: 4.4+ MB


CONVERT THE TARGET TO NUMERIC

In [10]:
df_lite["y"] = df_lite["y"].map({"no": 0, "yes": 1})

PREPARING FEATURE COLUMNS

In [11]:
# CATEGORICAL FEATURES
categorical_cols = ["default", "housing", "loan", "contact", "poutcome"]

# NUMERIC FEATURES
numeric_cols = ["campaign", "pdays", "previous", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]

# ISOLATE TARGET
X_lite = df_lite.drop(columns=["y"])
y = df_lite["y"]

# ENCODING CATEGORICAL VARIABLES
nominal_enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
preprocessor = ColumnTransformer(transformers=[("nominal", nominal_enc, categorical_cols), ("numeric", "passthrough", numeric_cols)])

# CREAT/APPLY THE FE PIPELINE
full_pipeline = Pipeline([ ("encode", preprocessor)])
X_transformed = full_pipeline.fit_transform(X_lite)

# WRANGLING FEATURE NAMES
nominal_feature_names = full_pipeline.named_steps["encode"].transformers_[0][1].get_feature_names_out(categorical_cols)
final_columns = list(nominal_feature_names) + numeric_cols

X = pd.DataFrame(X_transformed, columns=final_columns)


In [12]:
X.shape

(41188, 22)

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   default_no            41188 non-null  float64
 1   default_unknown       41188 non-null  float64
 2   default_yes           41188 non-null  float64
 3   housing_no            41188 non-null  float64
 4   housing_unknown       41188 non-null  float64
 5   housing_yes           41188 non-null  float64
 6   loan_no               41188 non-null  float64
 7   loan_unknown          41188 non-null  float64
 8   loan_yes              41188 non-null  float64
 9   contact_cellular      41188 non-null  float64
 10  contact_telephone     41188 non-null  float64
 11  poutcome_failure      41188 non-null  float64
 12  poutcome_nonexistent  41188 non-null  float64
 13  poutcome_success      41188 non-null  float64
 14  campaign              41188 non-null  float64
 15  pdays              

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

SCALING FOR KNN, LOGISTIC REGRESSION AND SUPPORT VECTOR MACHINE, DECISION TREE WILL JUST HAVE TO COME ALONG FOR THE RIDE.

In [14]:
# Split data into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y )


# Instantiate the scaler
scaler = StandardScaler()

# Fit on training data ONLY
scaler.fit(X_train[numeric_cols])

# Transform all sets
X_train[numeric_cols] = scaler.transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [15]:
print(f"X_train.shape--> {X_train.shape}, X_test.shape--> {X_test.shape}" )
print(f"y_train.shape--> {y_train.shape}, y_test.shape--> {y_test.shape}" )

X_train.shape--> (32950, 22), X_test.shape--> (8238, 22)
y_train.shape--> (32950,), y_test.shape--> (8238,)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [16]:
baseline_model = DummyClassifier(strategy='most_frequent', random_state=42).fit(X_train, y_train)
baseline_performance = baseline_model.score(X_test, y_test)
print(f"The scores to beat = baseline performance = {baseline_performance}; no is the common response")

The scores to beat = baseline performance = 0.8873512988589464; no is the common response


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [17]:
basic_model_lgr = LogisticRegression().fit(X_train, y_train)

### Problem 9: Score the Model

What is the accuracy of your model?

In [18]:
print(basic_model_lgr.score(X_test, y_test))

0.9009468317552805


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [19]:
def compare_models(model_set, return_best=0):
    best_models = {}
    results = []

    for name, (model, params) in model_set.items():
        # Create a pipeline
        pipeline = Pipeline([(name, model)])
        grid_search = GridSearchCV(pipeline, param_grid=params, cv=5, n_jobs=-1)

        # Fit the model and time it
        start_time = time.time()
        grid_search.fit(X_train, y_train)
        fit_time = (time.time() - start_time) / len(grid_search.cv_results_['mean_fit_time'])

        # Get the best estimator
        best_model = grid_search.best_estimator_
        best_models[name] = best_model              # <── store it

        # Evaluate on training and test sets
        train_score = best_model.score(X_train, y_train)
        test_score = best_model.score(X_test, y_test)

           # Append the results
        results.append([name, fit_time, train_score, test_score])

    # Create the results DataFrame
    results_df = pd.DataFrame(results, columns=['model', 'train time', 'train accuracy', 'test accuracy'])
    results_df.set_index('model', inplace=True)

    # Save to JSON (for submission if needed)
    results_df.to_json('data/model_results.json')
    df = pd.read_json(r'data/model_results.json')
    print(df)
    return best_models if return_best else None


In [20]:
# basic models with default hyperparameters
default_models = {
    'logisticregression': (LogisticRegression(), {'logisticregression__C': [1.0]}),
    'knn': (KNeighborsClassifier(), {'knn__n_neighbors': [5]}),
    'decisiontree': (DecisionTreeClassifier(), {'decisiontree__max_depth': [None]}),
    'svc': (SVC(), {'svc__C': [1.0], 'svc__kernel': ['linear']})
}
compare_models(default_models)

                    train time  train accuracy  test accuracy
logisticregression    2.975433        0.900061       0.900947
knn                   2.837421        0.909712       0.896334
decisiontree          1.807778        0.938816       0.889051
svc                 200.156393        0.897481       0.897669


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [21]:
# Pimproved models with some new choices for hyperparameters
improved_models = {
    'logisticregression': (LogisticRegression(max_iter=2000), {'logisticregression__C': [0.1, 0.5, 1.0]}),
    'knn': (KNeighborsClassifier(), {'knn__n_neighbors': [2, 4, 6, int(np.sqrt(len(X_train))/2)]}),
    'decisiontree': (DecisionTreeClassifier(), {'decisiontree__max_depth': [5, 7, 20]}),
    'svc': (SVC(), {'svc__C': [0.1, 0.5, 1.0], 'svc__kernel': ['rbf']})
}
best_improved_models = compare_models(improved_models, return_best=True)

                    train time  train accuracy  test accuracy
logisticregression    0.192110        0.899939       0.900825
knn                   1.097926        0.900364       0.901797
decisiontree          0.130260        0.902276       0.903253
svc                  22.450516        0.901517       0.902403


REVIEWING OTHER PERFORMANCE METRICS FOR THE IMPROVED MODEL SET

In [22]:
def create_model_metric_stack(model_set):
    eval_rows = []

    for name, model in fitted_models.items():
        if model is None:
            continue

        # Predictions
        y_pred = model.predict(X_test)
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)[:, 1]
        elif hasattr(model, "decision_function"):
            y_prob = model.decision_function(X_test)
        else:
            y_prob = None

        # Core metrics
        acc  = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, pos_label=1, zero_division=0)
        rec  = recall_score(y_test, y_pred, pos_label=1, zero_division=0)
        f1   = f1_score(y_test, y_pred, pos_label=1, zero_division=0)

        # Ranking metrics (skip if no probabilities)
        roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None
        pr_auc  = average_precision_score(y_test, y_prob) if y_prob is not None else None

        eval_rows.append([name, acc, prec, rec, f1, roc_auc, pr_auc])

    # results to DataFrame
    eval_df = pd.DataFrame(
        eval_rows,
        columns=["model", "accuracy", "precision", "recall", "f1", "roc_auc", "pr_auc"]
    ).set_index("model")

    print(eval_df.round(3))

In [23]:
fitted_models = {
    'baseline': baseline_model,
    **best_improved_models
}
create_model_metric_stack(fitted_models)

                    accuracy  precision  recall     f1  roc_auc  pr_auc
model                                                                  
baseline               0.887      0.000   0.000  0.000    0.500   0.113
logisticregression     0.901      0.725   0.193  0.305    0.791   0.455
knn                    0.902      0.688   0.235  0.350    0.795   0.454
decisiontree           0.903      0.669   0.279  0.394    0.798   0.444
svc                    0.902      0.685   0.248  0.364    0.684   0.381


CLOSING COMMENTS

Using the CRISP-DM framework, I predicted client term-deposit subscriptions from clean, but imbalanced bank-marketing data.
Four tuned classifiers—Logistic Regression, KNN, Decision Tree, and SVC—all exceeded the 0.887 baseline, achieving ~0.90 accuracy.
The Decision Tree performed best overall (test = 0.903, F1 = 0.394, ROC-AUC = 0.798), offering strong interpretability and balanced precision-recall.
Logistic Regression remained the most efficient (train < 0.2 s) and produced competitive precision (0.73) with solid ranking ability (ROC-AUC = 0.791).
Economic and campaign variables drove prediction power far more than demographics.
Next steps could include selecting a final model, refining hyperparameters, and exploring threshold adjustments or class-weighted, cost-sensitive objectives to improve recall while maintaining business value.

##### Questions